In [1]:
import pandas as pd
import numpy as np
from utils.Data import Data
from utils.Camera import Camera
from utils.Plot import Plot
import cv2

import plotly.graph_objects as go

In [2]:
data = Data()
camera = Camera('data/camera.dat')

INFO - Loading trajectory data from data//trajectory.dat
INFO - 0.00 [s] - Trajectory data loaded successfully!
INFO - Loading world data from data//world.dat
INFO - 0.01 [s] - World data loaded successfully!
INFO - Loading measurements data from data//meas-*.dat
INFO - 0.04 [s] - Measurements data loaded successfully!
INFO - Loading camera parameters from data/camera.dat
INFO - 0.00s - Camera parameters loaded successfully.


In [3]:
def data_association(measurement_1, measurement_2):
    gt_pose_1 = measurement_1['gt_pose']
    odom_pose_1 = measurement_1['odom_pose']
    points_1 = measurement_1['points']

    gt_pose_2 = measurement_2['gt_pose']
    odom_pose_2 = measurement_2['odom_pose']
    points_2 = measurement_2['points']

    matches = {'actual_point_id':[], 'appearance':[], 'image_point_1':[], 'image_point_2':[]}

    for i in range(len(points_1)):
        point_1 = points_1[i]
        actual_point_1 = point_1['actual_point_id']
        image_point_1 = point_1['image_point']
        appearance_1 = point_1['appearance']

        for j in range(len(points_2)):
            point_2 = points_2[j]
            actual_point_2 = point_2['actual_point_id']
            image_point_2 = point_2['image_point']
            appearance_2 = point_2['appearance']

            if appearance_1 == appearance_2:
                matches['actual_point_id'].append(actual_point_1)
                matches['appearance'].append(appearance_1)
                matches['image_point_1'].append(image_point_1)
                matches['image_point_2'].append(image_point_2)

    return gt_pose_1, odom_pose_1, gt_pose_2, odom_pose_2, matches

In [4]:
meas_0 = data.get_measurements(0)
meas_1 = data.get_measurements(1)

gt_pose_1, odom_pose_1, gt_pose_2, odom_pose_2, matches = data_association(meas_0, meas_1)
print('gt_pose_1', gt_pose_1)
print('odom_pose_1', odom_pose_1)
print('gt_pose_2', gt_pose_2)
print('odom_pose_2', odom_pose_2)
matches_df = pd.DataFrame(matches)
matches_df

gt_pose_1 [0.0, 0.0, 0.0]
odom_pose_1 [0.00160159, 0.0, -0.000259093]
gt_pose_2 [0.200426, 0.0, 0.0]
odom_pose_2 [0.200147, 0.00177571, 0.018434]


,actual_point_id,appearance,image_point_1,image_point_2
0,6,"[-0.668052, -0.119791, 0.76015, 0.658402, -0.3...","[522.119, 187.968]","[539.005, 183.622]"
1,14,"[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98...","[442.949, 142.838]","[463.689, 126.449]"
2,15,"[0.746543, 0.662075, 0.958868, 0.487622, 0.806...","[67.7619, 175.604]","[56.6646, 172.771]"
3,17,"[-0.439916, 0.0922137, 0.438537, -0.773439, -0...","[306.371, 119.738]","[305.329, 110.546]"
4,35,"[0.104633, 0.839182, 0.371973, 0.619571, 0.395...","[509.615, 216.308]","[518.356, 215.216]"
...,...,...,...,...
110,939,"[-0.426205, -0.741321, 0.567446, 0.887447, 0.2...","[265.905, 212.571]","[263.409, 211.305]"
111,963,"[-0.0250311, -0.80992, -0.385503, 0.413624, -0...","[302.141, 194.465]","[300.962, 191.459]"
112,971,"[0.683682, -0.00590533, 0.276129, -0.692478, 0...","[340.454, 198.249]","[341.861, 195.379]"
113,979,"[0.0400809, 0.470501, -0.1505, 0.457585, 0.206...","[110.052, 55.203]","[81.2067, 29.8137]"


In [5]:
data.print_trajectory(0)

Trajectory data: 
pose_id: 0
  ground_truth_pose: [0.0, 0.0, 0.0]
  odometry_pose:     [0.00160159, 0.0, -0.000259093]


In [6]:
data.print_world(6)

World data: 
landmark_id: 6
  landmark_position:   [2.79958, -2.91903, 0.751446]
  landmark_appearance: [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]


In [7]:
data.print_measurements(0)

Measurements data: 
sequence_id: 0
  ground_truth_pose: [0.0, 0.0, 0.0]
  odometry_pose:     [0.00160159, 0.0, -0.000259093]
  Points: 
    point_id: 0
      actual_point_id: 6
      image_point:     [522.119, 187.968]
      appearance:      [-0.668052, -0.119791, 0.76015, 0.658402, -0.339326, -0.542064, 0.786745, -0.29928, 0.37334, 0.912936]
    point_id: 1
      actual_point_id: 14
      image_point:     [442.949, 142.838]
      appearance:      [-0.80072, 0.0616159, 0.514588, -0.39141, 0.984457, 0.153942, 0.755228, 0.495619, 0.25782, -0.929158]
    point_id: 2
      actual_point_id: 15
      image_point:     [67.7619, 175.604]
      appearance:      [0.746543, 0.662075, 0.958868, 0.487622, 0.806733, 0.967191, 0.333761, -0.00548297, -0.672064, 0.660024]
    point_id: 3
      actual_point_id: 17
      image_point:     [306.371, 119.738]
      appearance:      [-0.439916, 0.0922137, 0.438537, -0.773439, -0.0570331, 0.18508, 0.888636, -0.0981649, -0.327298, 0.695369]
    point_id: 4
   

In [171]:
def Rt2T(R,t):
    T = np.eye(4)
    T[:3,:3] = R
    T[:3,3] = t.T
    return T

def initialize(meas_0, meas_1):

    Tc = camera.get_camera_transform()
    [z_near, z_far] = camera.get_camera_range()
    
    gt_pose_1, odom_pose_1, gt_pose_2, odom_pose_2, matches = data_association(meas_0, meas_1)

    set_1 = np.array(matches['image_point_1'])  
    set_2 = np.array(matches['image_point_2'])

    E, _ = cv2.findEssentialMat(set_1, set_2, camera.get_camera_matrix(), method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, set_1, set_2, camera.get_camera_matrix())
    T = Rt2T(R, t)

    P1 = np.dot(camera.get_intrinsic_matrix(), np.eye(4))
    P2 = np.dot(camera.get_intrinsic_matrix(), T)
    
    Q1_hom = cv2.triangulatePoints(P1, P2, set_1.T, set_2.T)
    Q2_hom = np.dot(T, Q1_hom)

    Q1_hom = np.dot(Tc, Q1_hom)
    Q2_hom = np.dot(Tc, Q2_hom)

    Q1 = (Q1_hom / Q1_hom[3])[:3]
    Q2 = (Q2_hom / Q2_hom[3])[:3]

    scale = np.mean(np.linalg.norm(Q2 - Q1, axis=0))

    world_points = Q2 * scale

    T = np.dot(Tc, T)


    world_points_gt = []
    for id in matches['actual_point_id']:
        world_point_gt = data.get_world(id)['landmark_position']
        world_points_gt.append(world_point_gt)

    return T, world_points.T, np.array(world_points_gt)
    
T, world_points, world_points_gt = initialize(meas_0, meas_1)
R = T[:3,:3]
t = T[:3,3]

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=[gt_pose_2[0]], y=[gt_pose_2[1]], z=[gt_pose_2[2]], mode='markers', marker=dict(size=3, color='red'))) 
fig.add_trace(go.Scatter3d(x=[t[0]], y=[t[1]], z=[t[2]], mode='markers', marker=dict(size=5, color='blue')))

for pw in world_points:
    fig.add_trace(go.Scatter3d(x=[pw[0]], y=[pw[1]], z=[pw[2]], mode='markers', marker=dict(size=2, color='orange')))

for pw in world_points_gt:
    fig.add_trace(go.Scatter3d(x=[pw[0]], y=[pw[1]], z=[pw[2]], mode='markers', marker=dict(size=2, color='green')))

fig.show()